In [1]:
import numpy as np
import pyvista as pv
from vtklb import vtklb
from clean_geometry import clean_geometry, mark_boundaries_solid, find_fluid_phase_clusters, calculate_normals
from time import time_ns


In [2]:
# ########################################################################## Load data
# ========================================================================== Data folders
data_path = "/home/AD.NORCERESEARCH.NO/esje/Programs/Python/LB/Data/"
output_path = "/home/AD.NORCERESEARCH.NO/esje/Programs/Python/CSSR/RelPerm/Data/"
# -------------------------------------------------------------------------- Solid
# pore >= 0 : SOLID
# pore  < 0 : FLUID
pore =  np.load(data_path + 'LV60A_PoreSolid_200x200x300_SDF.npy')
# -------------------------------------------------------------------------- Fluid
# fluid >= 0 : FLUID-PHASE 1
# fluid  < 0 : FLUID-PHASE 2
fluids = np.load(data_path + "LV60A_NWP_Sw062_200x200x300_SDF.npy") 
# -------------------------------------------------------------------------- geo
# SOLID         : 0
# FLUID PHASE 1 : 1
# FLUID PHASE 2 : 2
geo = np.ones(pore.shape, dtype=np.int32) 
# ========================================================================== Reduce system size
geo = geo[100:150, :50, :50]
fluids = fluids[100:150, :50, :50]
pore = pore[100:150, :50, :50]
# -------------------------------------------------------------------------- END Reduce system size

geo[fluids < 0] = 2 # mark fluid phase 2
geo[pore >= 0] = 0 # mark solid
# geo[:, 0, :] = 0
# geo[:, -1, :] = 0
# geo[0,: , :] = 0
# geo[-1, :, :] = 0
# ########################################################################## CLEAN GEOMETRY
# ========================================================================== Define lattice
# -------------------------------------------------------------------------- D3Q7
cc = np.ones((7, 3), dtype=np.int64)
cc[:4, :] = [
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1],
    [0, 0, 0]
    ]
cc[:-4:-1, :] = -cc[:3, :]
cc_d3q7 = np.copy(cc)
# -------------------------------------------------------------------------- D3Q19
cc = np.ones((19, 3), dtype=np.int64)
cc[:10, :] = [
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1],
    [1, 1, 0],
    [1, 0, 1],
    [0, 1, 1],
    [-1, 1, 0],
    [-1, 0, 1],
    [0, -1, 1],
    [0, 0, 0]
    ]
cc[:-10:-1, :] = -cc[:9, :]
cc_d3q19 = np.copy(cc)
del cc


In [3]:
# ========================================================================== Clean_geometry
t0 = time_ns()
while True:
    print("inside clean geo")
    geo_tmp = clean_geometry(cc_d3q7, cc_d3q19, geo)
    if np.all(geo_tmp == geo):
        break
    geo[:] = geo_tmp[:]
del geo_tmp 
t1 = time_ns()
print("    Run time for clean_geometry: ", (t1-t0)*1.0e-9)

# ========================================================================== Mark boundaries
t0 = time_ns()
geo_marked = mark_boundaries_solid(cc_d3q19, geo, 2)
t1 = time_ns()
print("    Run time for mark_boundaries: ", (t1-t0)*1.0e-9)

# ========================================================================== Find domains
t0 = time_ns()
domains, number_of_domains = find_fluid_phase_clusters(cc_d3q7, geo_marked, 2)
t1 = time_ns()
print("    Run time for find_fluid_phase_clusters: ", (t1-t0)*1.0e-9)

# ========================================================================== Set the force
t0 = time_ns()
# Remember the extended domains and flow in the z-direction
percolating_domains = set(domains[:, :, 1].flatten()) & set(domains[:, :, -2].flatten())
percolating_domains = percolating_domains - {0} # remove the solids
force = np.zeros_like(domains)
for n in percolating_domains:
    force[domains==n] = 1 
t1 = time_ns()
print("    Run time for set force: ", (t1-t0)*1.0e-9)

# ========================================================================== Isolated fluid domains
t0 = time_ns()
pressure_boundary_domains = set(domains[:, :, 1].flatten()) | set(domains[:, :, -2].flatten())
pressure_boundary_domains = pressure_boundary_domains - {0} # remove the solids
print(pressure_boundary_domains)
interior_domains = np.copy(domains)
# Set the domains that are in contact with a pressure boundary to zero
for n in pressure_boundary_domains:
    interior_domains[interior_domains==n] = 0
# shift the domain labels according to the removed domains
for n in sorted(pressure_boundary_domains, reverse=True):
    interior_domains[interior_domains > n] -= 1 
number_of_interior_domains = number_of_domains - len(pressure_boundary_domains)
t1 = time_ns()
print("    Run time for find isolated domains: ", (t1-t0)*1.0e-9)

# ========================================================================== Calculate normals
t0 = time_ns()
normals = np.zeros((3, ) + geo_marked.shape)
normals[:, 1:-1, 1:-1, 1:-1] = calculate_normals(fluids)
t1 = time_ns()
print("    Run time for calculate normals: ", (t1-t0)*1.0e-9)


inside clean geo
inside clean geo
    Run time for clean_geometry:  63.806227037000006
    Run time for mark_boundaries:  1.320565183
    Run time for find_fluid_phase_clusters:  2.6238291360000003
    Run time for set force:  0.060732314
{1, 3, 4, 7, 8, 18}
    Run time for find isolated domains:  0.183227777
    Run time for calculate normals:  3.9900846710000004


In [4]:
print(normals.shape)
print(domains.shape)

(3, 202, 202, 302)
(202, 202, 302)


In [5]:
# Write to vtk-file

geo = np.ones_like(geo_marked)
geo[:, :, 150:] = 2
# Solids (0) and ghost pressure boundaries (3) do not have 
# any additional boundary markers, so its safe to only test
# for 0 and 3 to set the solids in the geo file.
geo[geo_marked==0] = 0  
geo[geo_marked==3] = 0

# # Plot domains
# grid = pv.ImageData()
# grid.dimensions = geo.shape
# grid.origin = (0, 0, 0)
# grid.spacing = (1, 1, 1)
# grid.point_data["geo"] = geo.flatten(order="F")
# grid.plot(show_edges=False)

t0 = time_ns()
# ========================================================================== Write geometry
vtk = vtklb(geo, "D3Q19", "xyz", path="./VtkGeo/")
# ========================================================================== Write boundary information
vtk.append_data_set("nodetags", geo_marked)
# ========================================================================== Write the domain structure
vtk.append_data_set("domains", domains)
# ========================================================================== Write forcing
vtk.append_data_set("force", force)
# ========================================================================== Write interior domains
vtk.append_data_set("interior_domains", interior_domains)
# ========================================================================== Write normals
for n, val in enumerate(('x', 'y', 'z')):
    vtk.append_data_set("normal_" + val, normals[n])

t1 = time_ns()
print("   Run time for vtklb: ", (t1-t0)*1.e-9)

PERIODIC IN X

PERIODIC IN Y

PERIODIC IN Z

SYSTEM DEFINED BASIS D3Q19

Writing files to folder ./VtkGeo/
Wrote file: tmp0.vtklb
Wrote file: tmp1.vtklb
Appended data: nodetags to file tmp0.vtklb
Appended data: nodetags to file tmp1.vtklb
Appended data: domains to file tmp0.vtklb
Appended data: domains to file tmp1.vtklb
Appended data: force to file tmp0.vtklb
Appended data: force to file tmp1.vtklb
Appended data: interior_domains to file tmp0.vtklb
Appended data: interior_domains to file tmp1.vtklb
Appended data: normal_x to file tmp0.vtklb
Appended data: normal_x to file tmp1.vtklb
Appended data: normal_y to file tmp0.vtklb
Appended data: normal_y to file tmp1.vtklb
Appended data: normal_z to file tmp0.vtklb
Appended data: normal_z to file tmp1.vtklb
   Run time for vtklb:  291.94168402
